In [ ]:
import sqlite3
import requests
from bs4 import BeautifulSoup


response = requests.get('https://books.toscrape.com/')
soup = BeautifulSoup(response.text, 'html.parser')
soup

In [2]:
rate = {'One':1,
    'Two':2,
    'Three':3,
    'Four':4,
    'Five':5}
connection = sqlite3.connect('./DB/books.db')
for categories in soup.find('ul', attrs={'class':'nav nav-list'}).find('li').find('ul').find_all('a'):
    # category = categories.get_text().strip()
    link = 'https://books.toscrape.com/' + categories.get('href')
    books_response= requests.get(link)
    booksSoup = BeautifulSoup(books_response.text, 'html.parser')
    query_category= "SELECT id from Categories WHERE name LIKE ?"
    category_name = [booksSoup.find('div', attrs={'class':'page-header action'}).find('h1').get_text()]
    category_id = connection.execute(query_category,category_name).fetchall()[0][0]
    for book in booksSoup.find_all('article', attrs={'class':'product_pod'}):
        name = book.find('h3').find('a').get('title').strip()
        price = book.find('p', attrs={'class':'price_color'}).get_text()[2:]
        rating = rate[book.find('p').get('class')[1]]
        sql_insert = "INSERT INTO Books (name, price, rating, category_id) VALUES(?,?,?,?)"
        values = [name, price, rating, category_id]
        cursor = connection.execute(sql_insert,values)
        connection.commit()
    while((booksSoup.find('li', attrs={'class':'next'})) != None):
        link =link.rsplit('/', 1)[0] + "/" + booksSoup.find('li', attrs={'class':'next'}).find('a').attrs['href']
        next_pageresponse= requests.get(link)
        booksSoup = BeautifulSoup(next_pageresponse.text, 'html.parser')
        for book in booksSoup.find_all('article', attrs={'class':'product_pod'}):
            name = book.find('h3').find('a').get('title').strip()
            price = book.find('p', attrs={'class':'price_color'}).get_text()[2:]
            rating = rate[book.find('p').get('class')[1]]
            sql_insert = "INSERT INTO Books (name, price, rating, category_id) VALUES(?,?,?,?)"
            values = [name, price, rating, category_id]
            cursor2 = connection.execute(sql_insert,values)
            connection.commit()
connection.close()
           
                    
            

